In [13]:
import pandas as pd

from data_cleaning_py import fetch_and_clean_data

tetouan = fetch_and_clean_data()

tetouan.head()

c:\Users\bfran\Ironhack\Week 7\powerhouse-squad-project\data_cleaning_py.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # metadata
c:\Users\bfran\Ironhack\Week 7\powerhouse-squad-project\data_cleaning_py.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # variable information


,DateTime_cleaned,Temperature,Humidity,Wind Speed,DayOfWeek_Monday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,DayOfWeek_Thursday,DayOfWeek_Friday,DayOfWeek_Saturday,DayOfWeek_Sunday,TimeOfDay_Morning,TimeOfDay_Afternoon,TimeOfDay_Evening,TimeOfDay_Night,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,2017-01-01 00:00:00,6.559,73.8,0.083,False,False,False,False,False,False,True,False,False,False,True,34055.69620,16128.87538,20240.96386
1,2017-01-01 00:10:00,6.414,74.5,0.083,False,False,False,False,False,False,True,False,False,False,True,29814.68354,19375.07599,20131.08434
2,2017-01-01 00:20:00,6.313,74.5,0.080,False,False,False,False,False,False,True,False,False,False,True,29128.10127,19006.68693,19668.43373
3,2017-01-01 00:30:00,6.121,75.0,0.083,False,False,False,False,False,False,True,False,False,False,True,28228.86076,18361.09422,18899.27711
4,2017-01-01 00:40:00,5.921,75.7,0.081,False,False,False,False,False,False,True,False,False,False,True,27335.69620,17872.34043,18442.40964
